# Bot simples de Halite IV

## Conteúdos
* Código base para desenvolvimento de bots no Halite IV
* Explicações sobre a linguagem python
* Funcionalidades da biblioteca de ambiente utilizada pelo jogo.

### Variáveis em Python
As variáveis em python não possuem um único tipo definido, ao invés disso, uma mesma
variável pode mudar de tipo durante a execução do programa sem problemas. Essas variáveis
são declaradas simplesmente ao se atribuir valor a um nome de variável ainda não existente.
O texto na mesma linha e após o '#' é um comentário
```python
var = 2 # Declara a variável "var" como o número 2
var = "Alguma string" # Muda o tipo de "var" para string.
```

In [ ]:
# Configura o ambiente disponibilizado pelo Kaggle do Halite IV
from kaggle_environments import evaluate, make
# Declara a variável que armazena informações sobre o ambiente do jogo
env = make("halite", configuration={ "episodeSteps": 400 }, debug=True)

# Imprime as configurações do ambiente
print (env.configuration)

### Escopo em python
Em muitas linguagens de programação o escopo de uma função, ou de um laço de repetição é definido
pelas chaves {}. Em python isso é feito através da identação, ou seja, para saber se um código se
encontra dentro ou fora de um escopo, basta verificar sua identação (número de espaços do início da
linha até o primeiro caractere que não seja espaço).
De maneira simples, se tudo estiver alinhado verticalmente conforme o que se espera, a identação, e
portanto os escopos, estarão definidos corretamente.

### Funções em python
Em python as funções são definidas com a palavra **"def"** seguido do nome da função.
Diferentemente de outras linguagens de programação, uma função em python não tem tipo definido,
isso significa que ela pode retornar qualquer tipo de dados e cabe ao programador saber qual/quais
tipo(s) pode(m) ser retornado(s) (para retornar um valor, se usa da palavra **"return"**).
Além disso, funções podem receber argumentos que devem ser listados dentro de parênteses (também sem
tipo definido).

#### Alguns exemplos de funções
```python
def soma (a, b):
    # Repare na identação que indica que esse return faz parte da função soma
    return a + b

# Função de multiplicação de dois números naturais através de somas consecutivas.
def mult(a, b):
    # If não precisa de parêntesis.
    # Vale notar que em python, não existe o "else if" e ao invés se usa "elif" (mesma funcionalidade)
    if b == 0:
        return 0
    else:
        # Demonstra a capacidade recursiva da linguagem
        return soma(a, mult(a, b-1))
```

## Laços de repetição em python
Assim como em muitas outras linguagens de programação, em python também existem os laços **"for"** e
**"while"**. O "while" funcona de forma muito similar, mas o "for" possui certas mudanças.
```python
# Em python, os valores booleanos são True e False (capitalizados)
while True:
    # Loop infinito
    print("infinito")

# Itera por uma série de 10 números que vão de 0 a 9 (função range(10))
for i in range(10):
    # Imprime na tela "Contador: 0" na primeira iteração, "Contador: 1" na segunda, etc.
    # Operador + concatena (junta) a string "Contador: " com a string do número armazenado em i
    # (str(i))
    print("Contador: " + str(i))

# Itera por cada um dos números da lista e os imprime
for fib in [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]:
    print("Fibonacci: " + str(fib))
```

## Papel do agente
No código podem haver muitas definições de função, mas existe uma restrição apenas sobre a última função declarada:
* **A última função deve receber dois argumentos:**
    * Uma observação do ambiente, isto é, a posição de todos os inimigos, minérios, etc (o estado do jogo).
    * Uma configuração que contém metadados sobre a partida, isto é, quantos turnos até o final do jogo, etc.
* **A última função deve retornar a ação a ser executada pelo agente:**
    * Um navio pode:
        * Mineirar, isto é, ficar parado.
        * Andar uma casa para o norte, o sul, o leste ou o oeste.
        * Se transformar num estaleiro.
    * Um estaleiro pode:
        * Criar um novo navio.
        * Não fazer nada.

Essa última função será executada a cada turno do jogo e será a única forma de interação direta com o jogo.

In [ ]:
%%writefile submission.py

from kaggle_environments.envs.halite.helpers import *
from random import choice

# Função principal do agente: recebe observação e configuração e retorna uma ação.
def agent(obs, config):
    # Constrói um objeto Board a partir da observação e configuração.
    board = Board(obs, config)
    # Atribui à variável "me" o objeto do jogador da vez (no caso, o próprio agente)
    me = board.current_player
    
    # Itera por todos os navios disponíveis para o agente
    for ship in me.ships:
        # Configura a próxima ação do navio para ser uma escolha aleatória entre norte, sul, leste, oeste ou não fazer nada.
        ship.next_action = choice([ShipAction.NORTH, ShipAction.EAST, ShipAction.SOUTH, ShipAction.WEST, None])
    
    # Itera por todos os estaleiros disponíveis para o agente
    for shipyard in me.shipyards:
        # Define que a próxima ação deve ser não fazer nada.
        shipyard.next_action = None
    
    # Retorna todas as ações definidas para cada um dos navios e estaleiros.
    return me.next_actions

In [ ]:
# Utiliza do ambiente do jogo para rodar o bot contra outros 3 agentes aleatórios
env.run(["/kaggle/working/submission.py", "random","random","random"])
# Renderiza uma caixa de visualização com o jogo logo abaixo
env.render(mode="ipython", width=800, height=600)

# É possível ver as ações do bot no jogo abaixo (bot amarelo)